# Final Project Report

Team members: Xiao Ji (xj2247@columbia.edu, xj2247), Xinyi Liu (xl2904@columbia.edu, xl2904), Jiaying Chen (jc5299@columbia.edu, jc5299), Duanyue Yun (dy2400@columbia.edu, dy2400)

# 1. Objectives

For regular recommendation systems, previous ratings of users are the most important factor to consider. Using these ratings, we can infer user preferences and recommend items to users based on users' previous behavior. Matrix factorization is one of recommendation models for recommending similar items to users, but by only considering users' previous ratings, we may miss a lot of information about users and items which might lead to better recommendations. In order to enhance the precision of recommendation, we should take into account information related to users and items. However, not every feature of users or items is useful due to some features may lead to overfitting or underfitting. In this case, what we need to do when building a recommendation model is to select features and then use these features and previous ratings to predict a user's latest rating or recommendate top K items to users.

There are two models that consider side information of users and items, LightFM and CMF. We build these two models to evaluate their quality, and compare these two models to baseline (bias and regular matrix factorization). Besides, we are also interested in the quality of models on different segments of users or on different segments of items (with different level of activity).

# 2. Data Analysis

There are 6 datasets in Yelp Dataset: `business.json`, `review.json`, ` user.json`, `checkin.json`, `tip.json`, `photo.json`.

Becuase we want to use the side information of users and items and the previous ratings to recommend, we only use three datasets in Yelp Dataset:
* `review.json`: it contains full review text data, including user_id, business_id and ratings
* `business.json`: it contains business data, including business_id and side information of business
* `user.json`: it contains user data, including user_id and side information of user

## 2.1 Users



Atfer filtering very inactive users whose number of reviews in Yelp are below 5, there are 286130 users in `review` dataset. In order to evaluate how the recommendation system works for different users, we analyzed the distribution of review count of users so that we can divide users into three segments, active, moderate and inactive.

From the boxplot of review count of users, it's surprising that there are several outliers, the maximum review count is up to 4000, and most of users have fewer than 20 reviews in Yelp. What's more, 25-th percentile users have 6 reviews, 50-th percentile users have 8 reviews, and 75-th percentile users have 15 reviews. 

<img src="./images/users_distribution.png" width="800" align="center"/>
<figcaption><center>Users Distribution</center></figcaption>

Because outliers are too far from the upper fence, it's difficult to display the details of distribution of majority of users. Thus, we select users with fewer than 100 reviews to show the distribution.

<img src="./images/users_hist.png" width="500" align="center"/>

The histogram of users shows a long-tail distribution, and based on the boxplot, we divide users into three segments:
* `Active users`: users with 20 reviews and more
* `Moderate users`: users with 10 reviews and more but fewer than 20 reviews
* `Inactive users`: users with fewer than 10 reviews

And the dataset yields $47944$ active users, $74880$ moderate users and $163306$ inactive users.

## 2.2 Items

There are 185723 businesses in review dataset.

In the boxplot, it's clear that there are also a lot of outliers, and the maxmium review counts of item is 4672, while most of items only have fewer than 40 reviews. Besides, we can know that 25-th percentile items have 3 reviews, 50-th percentile items have 6 reviews and 75-th percentile items have 18 reviews.

<img src="./images/items_distribution.png" width="800" align="center"/>
<figcaption><center>Items Distribution</center></figcaption>

We select items with fewer than 350 reviews to show the distribution of items.

<img src="./images/items_hist.png" width="500" align="center"/>

After examining distribution of review counts of items, we divide items into 3 segments based on the following:

* `popular items`: items with 300 reviews and more
* `moderate items`: items with 10 reviews and more but fewer than 300 reviews
* `unpopular items`: items with fewer than 10 reviews

And the dataset yields $1844$ very popular items, $70549$ average items and $113330$ unpopular items.

## 2.3 Features

We examined the business.json and user.json files to look for possibly useful side information. 

For users, we computed a variable `year` that denotes the number of years the user has been using Yelp. We did not use variables like the number of compliments or the number of fans because even the 75-th percentile is 0, which means majority of users did not receive any compliments or have any fans. Therefore, we think such information might not be very useful.

<img src="images/user_attr.png" width="700" align="center"/>
<figcaption><center>Most variables in user.json file take the value zero.</center></figcaption>

For items, we considered `state`, `review count`, `stars` and `categories` to be potentially useful item attributes. After some data exploration, we found that the number of businesses in each state is different and the average rating by state is also slightly different. Therefore, we include state as part of item side information. 
Also, the categories that a business belongs to also imply a user's preferences and might help with the recommendation task.  

<img src="images/item_attr.png" width="200" height = "250" align="center"/>
<figcaption><center>Summary statistics of businesses by state (first 8 rows)</center></figcaption>

# 3. Base Line

## 3.1 Empirical Bias Model
Formula: $\widehat{r_{ij}} = r + b_i^{user}+ b_j^{item}$   
This is the empirical baseline model that measured item and user bias for each entry. It only computes predicted rating by adding overall average rating, user bias and item bias. Here we use RMSE as our accuracy metric to evaluate the models.

To see the effectiveness of our models using side information, we built two baseline models that simply use rating records. One is the empirical bias model and another is matrix factorization Model.  
Then we divided users into active, moderate and inactive; items into popular, moderate and unpopular and compared accuracy across different segments.

First we calculate the overall RMSE of the bias baseline model, which is 1.392. And then we used the similar code to calculate the RMSE in each group. Then we get the RMSE results.

- Bias baseline RMSE: 1.392

- Bias baseline RMSE (not so active users): 1.437
- Bias baseline RMSE (average users): 1.357
- Bias baseline RMSE (very active users): 1.282
   
- Bias baseline RMSE (unpopular items): 1.641
- Bias baseline RMSE (moderate items): 1.38
- Bias baseline RMSE (popular items): 1.249

## 3.2 Matrix Factorization Model
Matrix Factorization is the collaborative filtering algorithms that we used in the Project 1. Specifically, we use the Alternating Least Square (ALS) algorithm with Spark ML. Since we do not include any side information in this model, we used it as one of our baseline model. In this part, we also train the model by Spark ML.

Additionally, since we build the model of Factorization Machine using the lightFM package (provide metrics like AUC and precision), we also build another baseline using lightFM with no side information in later parts.

### MF model by Spark ML

We built the Matrix Factorization Model by Spark ML. We used 5-fold cross validation to tune the following hyperparameters:
1. number of latent factors: 5 to 15 with intervals of 5; 
2. maximum number of iterations: 5 to 15 with intervals of 5; 
3. regularization parameter: 0.01, 0.1, 0.5

Using the CrossValidator function in Spark ML, we can get the best parameters map:  
**Best Model**
Rank:  10
MaxIter:  15
RegParam:  0.5  

Similarly, we first we calculate the overall RMSE of the MF model, which is 1.4370287007673046. And then we used the similar code to calculate the RMSE in each group. Then we get the RMSE results:

- MF baseline RMSE: 1.4370287007673046
- MF baseline RMSE (not so active users): 1.4775870789989252
- MF baseline RMSE (average users): 1.4070938148225576
- MF baseline RMSE (very active users): 1.3399711012473254
- MF baseline RMSE (unpopular items): 1.650259185689958
- MF baseline RMSE (moderate items): 1.431661381240479
- MF baseline RMSE (popular items): 1.315975732947149

The two baseline models would serve as a benchmark for us to evaluate our models that incorporate side information (CMF, FM).

# 4. Model

In order to beat the baseline, we tried two approaches. One is collective matrix factorization using the `cmfrec` package. The other one is factorization machine using the `lightFM` package.

## 4.1 Collective Matrix Factorization

### Side Information

For CMF, we decide to use `year` that denotes the number of years the user has been using Yelp as user attributes. As for item attributes, since we have a number of options, we did some model exploration and found that including `state` only gives the highest accuracy. Therefore, we only used the `state` column as item attributes. And in `cmfrec` package, the model by default considers item bias and user bias, so there is no need to include `stars` and `review counts` as item attributes. `categoty` is a string variable, and there are too many unique values (in thousands) in `categoty`. So we did not include `category` either.

### Model Optimization

For hyper-parameter tuning, we randomly sampled 1% of the training dataset to save time in computation. Therefore, we used $33664$ unique users and $25835$ unique items for model optimization. After we obtain the optimal parameters, we fit the model on the entire training dataset and compute RMSE on the held-out test data which contain the last review of each user. 

To optimize the model, we took some exploratory steps by trying different values of $k$, which is the number of common latent factors to use. We tried $k=20, 40, 60$, but this significantly increased the running time without much improvement in RMSE (the change is less than 0.0001). Therefore, we decide to use the default value $k=30$ provided by the package and assigned different weights to the main rating matrix, user information and item information. Also, we were unable to run cross validation because of time constraints. Below is the set of model specifications that we used and the corresponding RMSE value.

<img src="images/params.png" width="280" align="center"/>
<figcaption><center>Model specifications and performance</center></figcaption>

<img src="images/pcp.png" width="800" align="center"/>
<figcaption><center>Parallel coordinate plot of parameters and RMSE</center></figcaption>

The model that yields the lowest RMSE is $w_{main} = 1$, $w_{user} = 1$ and $w_{item} = 0$.

With the optimal weights, we fit the model on the entire training dataset and compute RMSE on the test data. 

Later we discovered that increasing the weights help lower RMSE. Since the ratio of the weights remain the same, we still have the equivalent model specification. The results are shown below.

<img src="images/weights.png" width="280" align="center"/>
<figcaption><center>Integer weights used and corresponding RMSE.</center></figcaption>

<img src="images/rmse_change.png" width="400" align="center"/>
<figcaption><center>Change in RMSE with weights</center></figcaption>

Since using integer weight 9 yields the lowest RMSE, we used that as our final model specification.

## 4.2 Model 2 - Factorization Machine from LightFM

We used LightFM package to build the factorization machine model with both user and item features. The details about LightFM can be found here: https://lyst.github.io/lightfm/docs/home.html

### 4.2.1 Feature selection

We randomly selected 1% users to test different combinations of features. The features we tested are described in section **2.3**: **state**, **stars**, **review_count** and **categories** are item features, and **yelping_since (year)** are user features.

We randomly selected 1% user_ids and extracted the relevant records from ratings, business and users datasets:

In [ ]:
import random 
random.seed(12345)
sample_1pc = random.sample(active_users,int(0.01*len(active_users)))
sample_ratings=ratings.loc[ratings['user_id'].isin(sample_1pc)]
sample_business=business.loc[business['business_id'].isin(sample_ratings['business_id'])]
sample_users=users.loc[users['user_id'].isin(sample_ratings['user_id'])]

number of **users** in the sample:  2861  
number of **businesses** in the sample:  28191  
number of **ratings** in the sample:  49133  

Then we splitted the sample into 80% training and 20% test, and run LightFM using different combinations of features mentioned above. Below is the summary of the results:

| Feature Combination | Training precision@10 | Training AUC | Test precision@10 | Test AUC
| :--- | :--- | :--- | :--- | :--- |
| stars + state + review_counts + year | 0.0283 | 0.93 | 0.0052 | 0.89
| stars + state + review_counts + categories | 0.0312 | 0.94 | 0.0051 | 0.88
| stars + state + review_counts | 0.0274 | 0.94 | 0.0062 | 0.88
| stars + state | 0.0329 | 0.95 | 0.0050 | 0.88
| stars + review_counts | 0.0171 | 0.86 | 0.0023 | 0.62
| state + review_counts | 0.0292 | 0.94 | 0.0050 | 0.88


The codes and full outputs of feature selection can be found in **Feature selection.ipynb**

**stars + state + review_counts** yields the best test precision@10 and **stars + state + review_counts + year** yields the best test AUC. We will keep the feature **year** because we want to include at least one user feature in our model. We believe that when sample size grows larger, the user feature will be helpful. Also, we will use regularization parameter in the full dataset to avoid overfitting.  
  
One can also find that including **categories** in the model will improve the training result but decrease the test result, which may be due to overfitting. Also, removing **state** significantly decreases both precision and AUC, so **state** is an important item feature in the factorization machine model.  

In conclusion, we will use **stars + state + review_counts + year** to do the cross validation (4.3.2) and the full dataset modeling (4.3.3).

### 4.2.2 Tuning hyper-parameters by cross validation 

Again, we used 1% users to do the 2-fold cross validation. The hyper-parameters we tuned include:  
**no_components**: the dimensionality of the feature latent embeddings.  - [10, 20, 30]  
**loss**: the loss function.  – ['bpr’, ‘warp’]  
**item_alpha**: L2 penalty on item features.  – [0, 0.0001]  
**user_alpha**: L2 penalty on user features.  – [0, 0.0001]  
**epochs**: number of epochs to run. - [10, 20, 30]  

The codes and full outputs of cross validation can be found in **lightFM cross validation.ipynb**.  

Below are the two hyper-parameter combinations that give the best AUC and precision@10:  

In [17]:
cv_result.loc[cv_reult['AUC'] == cv_result['AUC'].max(),]

,loss,no_components,num_epochs,item_alpha,user_alpha,precision_10,AUC
61,warp,30,10,0.0,0.0001,0.009866,0.883216


In [19]:
cv_result.loc[cv_result['precision_10'] == cv_result['precision_10'].max(),]

,loss,no_components,num_epochs,item_alpha,user_alpha,precision_10,AUC
67,warp,30,20,0.0001,0.0001,0.0102,0.879504


We have tried two optimization method:   
**WARP** (Weighted Approximate-Rank Pairwise): Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found; and   
**BPR** (Bayesian Personalised Ranking pairwise loss): Maximises the prediction difference between a positive example and a randomly chosen negative example.    
reference: https://lyst.github.io/lightfm/docs/lightfm.html  
**WARP** gives the better results while keeping other hyper-parameters same.


Although we could not achieve the best precision and AUC at the same time, these two combinations yield similar results. We will use the second combination (67) for two reasons:  
1. It is very likely to overfit the model when we introduce additional interaction terms, so we prefer to add both user and item regularizations.  
2. We care more about precision@10 because in practice the system will only recommend limited number of items to a user, so the top few items instead of the whole ranking matters.

### 4.2.3 Modeling the full dataset

The modeling process can be divided into the following steps:

#### Step 1. Select the most recent rating as test, the rest as training

In [ ]:
ratings_test = act_ratings.groupby('user_id').tail(1)
ratings_training = act_ratings.drop(ratings_test.index)

number of **users** in the training:  286130  
number of **business** in the training:  183637  
number of **ratings** in the training:  4252142  
number of **users** in the test:  286130  
number of **business** in the test:  45788  
number of **ratings** in the test:  286130  

#### Step 2. Build the dataset and fit the user/item id and feature name mappings.

In [ ]:
dataset = Dataset()
dataset.fit((act_ratings['user_id']),
            (act_ratings['business_id']))
dataset.fit_partial(items=(act_business['business_id']),
                    item_features = (act_business['stars']))

dataset.fit_partial(items=(act_business['business_id']),
                    item_features = (act_business['state']))

dataset.fit_partial(items=(act_business['business_id']),
                    item_features = (act_business['review_count']))

dataset.fit_partial(users=(act_users['user_id']),
                    user_features = (act_users['year']))

#### Step 3. Build training and test  interactions seperatly as well as the feature interactions

In [ ]:
# training interactions
(interactions_training, weights) = dataset.build_interactions((ratings_training['user_id'][i],ratings_training['business_id'][i]) 
                                                     for i in range(len(ratings_training)))

# test interactions
(interactions_test, weights) = dataset.build_interactions((ratings_test['user_id'][i],ratings_test['business_id'][i]) 
                                                     for i in range(len(ratings_test)))

# feature interactions
item_features = dataset.build_item_features(((act_business['business_id'][i], [act_business['stars'][i],
                             act_business['state'][i],act_business['review_count'][i]])
                                              for i in range(len(act_business))))

user_features = dataset.build_user_features(((act_users['user_id'][i], [act_users['year'][i]])
                                              for i in range(len(act_users))))

#### Step 4. Fit the LightFM model with and without features
The LightFM model without features is just the regular matrix factorization model. We can use it as the baseline to check if adding feature interactions improves the result.

In [ ]:
# LightFM with features
model1 = LightFM(loss='warp',no_components=30, item_alpha=0.0001, user_alpha=0.0001)
model1.fit(interactions_training,epochs=20,item_features=item_features,user_features=user_features)

In [ ]:
# LightFM without features
model2 = LightFM(no_components=30)
model2.fit(interactions_training,epochs=20)

#### Step 5: Evaluate the results

In [14]:
# precision at 5
FM_precision_overall = precision_at_k(model1,interactions_test, train_interactions = interactions_training, check_intersections=False,
                                      item_features=item_features, user_features=user_features,k=5).mean()
print(FM_precision_overall)

BL_precision_overall = precision_at_k(model2,interactions_test,train_interactions = interactions_training, check_intersections=False, k=5).mean()
print(BL_precision_overall)

0.00067521754
7.7587116e-05


In [14]:
# precision at 5
FM_precision_overall = precision_at_k(model1,interactions_test, train_interactions = interactions_training, check_intersections=False,
                                      item_features=item_features, user_features=user_features,k=5).mean()
print(FM_precision_overall)

BL_precision_overall = precision_at_k(model2,interactions_test,train_interactions = interactions_training, check_intersections=False, k=5).mean()
print(BL_precision_overall)

0.00067521754
7.7587116e-05


Precicion@5 means the fraction of the top 5 recommended items (the known interactions in the train set will be omitted) that contains the known positives. 0.2 will be the perfect precision@5 here because the only positive in the test data is the last rated restaurant, so the best will be the top 5 recommended items contain the last rated restaurant and the precision will be 1/5=0.2. We then average the precision for all users to get the final precision. We used precision@5 here because we only target one restaurant and we don't want the range of recommended list to be too large.  

We can see that both precision@5 and AUC improve a lot when we add feature interactions. We then segmented the users and items into active/moderate/non-active and popular/moderate/unpopular and calculated their precision and AUC respectively, below is the summary of the results:

**precision@5 (*0.001)**

|  | baseline - MF (lightFM) | FM (lightFM)
| :--- | :--- | :--- |
|overall|	0.078|	0.675
|active users|	0|	0.117
|moderate users|	0.005|	0.334
|non active users|	0.133|	0.783
|popular items|	0.325|	2.298
|moderate items|	0|	0.007
|unpopular items|	0|	0

**AUC**

|  | baseline - MF (lightFM) | FM (lightFM)
| :--- | :--- | :--- |
|overall|	0.818|	0.949
|active users|	0.806|	0.961
|moderate users|	0.815|	0.953
|non active users|	0.822|	0.944
|popular items|	0.992|	0.996
|moderate items|	0.845|	0.962
|unpopular items|	0.247|	0.766

Adding users and items features improve the overall results as well as the results by segment. Details about the code can be referred to:

**lightFM full dataset - overall results.ipynb**  
**lightFM full dataset - precision by segment.ipynb**  
**lightFM full dataset - auc by segment.ipynb**.

# 5. Comparing Models

## 5.1 Accuracy metrics

|  RMSE| baseline - bias | baseline - MF (ALS)|CMF
| :--- | :--- | :--- | :--- |
|overall|	1.392|	1.347| __1.367__
|active users|	__1.282__|	1.340| 1.287
|moderate users|	__1.357__|	1.407| 1.362
|non active users|	1.437|	1.478| __1.393__
|popular items|	1.249|	1.316| __1.215__
|moderate items|	1.380|	1.432| __1.366__
|unpopular items|	1.641|	1.650| __1.589__

|  Precision(* 0.001)| baseline - MF (cmfrec) | baseline - MF (lightFM)|FM (lightFM)| CMF
| :--- | :--- | :--- | :--- | :--- |
|overall|	0.121|	0.078| __0.675__| 0.294
|active users|	0.142|	0.000| 0.117| __0.150__
|moderate users|	0.110|	0.005| __0.334__| 0.323
|non active users|	0.120|	0.133| __0.783__| 0.323
|popular items|	0.454|	0.325| __2.298__| 1.268
|moderate items|	0.043|	0.000| 0.007| __0.054__
|unpopular items|	0.000|	0.000| 0.000| 0.000

|  AUC| baseline - MF (lightFM)|FM (lightFM)
| :--- | :--- | :--- |
|overall|	0.818|	__0.949__|
|active users|	0.806|	__0.961__|
|moderate users|	0.815|	__0.953__|
|non active users|	0.822|	__0.944__|
|popular items|	0.992|	__0.996__|
|moderate items|	0.845|	__0.962__|
|unpopular items|	0.247|	__0.766__|

<table><tr>
<td> <img src="./images/RMSE.png" alt="Drawing" style="width: 390px; height: 250px"/> </td>
<td> <img src="./images/AUC.png" alt="Drawing" style="width: 390px; height: 250px"/> </td>
<td> <img src="./images/precision.png" alt="Drawing" style="width: 390px; "/> </td>
</tr></table>

1. By general comparison, we can see that:
    - FM and CMF model both beat the baseline in terms of accuracy.
    - By comparison in precision-at-5, the FM Model can recommend better than the CMF model in most groups.
    
  
2. By RMSE comparison, we can see that:
    - RMSE of Collective Matrix Factorization is less than the RMSE of the baseline model, except for group "active users" and "moderate users". But the difference is small.
    - For all models, as the active level of users or popularity of business increases, RMSE decreases. This suggests they can recommend better for active users and popular businesses.
    

3. By AUC and precision-at-5 comparisons, we can see that:
    - FM model has greater improvement in recommendation while the CMF model is still better than two baseline models.
    - According to AUC comparison, the matrix factorization model might not be able to recommend well in group "unpopular items". The effectiveness of this model also decreases as the popularity of items decreases.
    - The FM model has better stability since there are no significant differences among groups. It also shows that recommendations for unpopular items has been significantly improved compared to the MF model.
    
    
4. Comparing the precision of Matrix Factorization models run by cmfrec and lightFM package:
    - As can be seen from the first two columns of the second table, the precisions are similar for non active users and popular items, and the MF model run by cmfrec package gives a slightly better overall result. The reason may be that different packages use different optimization functions and the number of iterations could also be different. In order to further study the matrix factorization model in different packages, one should set all the parameters and methods to be the same. We will not dig into it in this project.

## 5.2 Catalog Coverage

Beyond accuracy metrics, we also computed catalog coverage to evaluate the two models we built. We define catalog coverage as the fraction of items that are in the top-5 for at least 1 user. There are 185,723 distinct items in the review dataset, so the CMF model covers roughly 115 distict restaurants in the top 5 list for all users and the FM model covers roughly 123 distinct restuarants. The FM model using lightFM has slightly higher catalog coverage. This further confirms that FM is better than CMF in terms of coverage.


|  Model| Catalog coverage 
| :--- | :--- 
|CMF (cmfrec)|	0.00061920
|FM (lightFM)|	__0.00065689__


# 6. Conclusion

In conclusion, we built two models (CMF and FM) and beat the two baseline models (bias and regular MF) in terms of accuracy. In general, the models are more accurate for more active users and more popular items. When comparing CMF and FM, we find that FM performs better even when we use a non-accuracy metric like catalog coverage. Further improvements we can make is to use time aware models that take into account change in user preferences over time.